В данном блокноте удаляю дубликаты session_id, причём те, у которых target_event = 0

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from warnings import filterwarnings

filterwarnings('ignore')

%store -r df

# для этого датафрейма 
#df = df.sample(frac=1).reset_index(drop=True)
#df = df.sample(n=50000).reset_index(drop=True)

res = df[~((df.duplicated(['session_id'])) & (df['target_event']==0) )]
for col in res.columns:
    res[col] = res[col].fillna('unknown')

## visit_date и visit_time
здесь делаю некоторые манипуляции с датой и временем: смотрю сколько было целевых действий за месяц и за час - оказывается, час и месяц влияют на целевое действие

In [2]:
%%time
res['visit_time'] = pd.to_datetime(res['visit_time'])
res['visit_hour'] = res['visit_time'].apply(lambda x: int(x.hour))
vc = res[res['target_event']==1]['visit_hour'].value_counts()

ranges_hour = [0, 1000, 2000, 3000, 4000]
cats_hour = [1, 2, 3, 4]
df_vc = pd.DataFrame(vc)
df_vc['hour'] = df_vc.index
df_vc = df_vc.rename(columns={'visit_hour': 'count'})
df_vc.reset_index(drop=True)
df_vc['cat'] = pd.cut(df_vc['count'], ranges_hour, labels=cats_hour)

res['cat_hour'] = res['visit_hour'].apply(lambda x: df_vc.loc[df_vc['hour'], 'cat'][x])

res['visit_date'] = pd.to_datetime(res['visit_date'])
res['visit_month'] = res['visit_date'].apply(lambda x: int(x.month))

vcm = res[res['target_event']==1]['visit_month'].value_counts()

ranges_month = [0, 5000, 6000, 7000, 8000]
cats_month = [1, 2, 3, 4]
df_vcm = pd.DataFrame(vcm)
df_vcm['month'] = df_vcm.index
df_vcm = df_vcm.rename(columns={'visit_month': 'count'})
df_vcm.reset_index(drop=True)
df_vcm['cat'] = pd.cut(df_vcm['count'], ranges_month, labels=cats_month)
res['cat_month'] = res['visit_month'].apply(lambda x: df_vcm.loc[df_vcm['month'], 'cat'][x])

res = res.drop(['visit_date', 'visit_month', 'visit_time', 'visit_hour'], axis=1)

CPU times: total: 11min
Wall time: 11min 1s


## UTM_*
### utm_source

In [3]:
def foo_source(x):
    #print(x)
    try:
        t = df_vc_utmsource.loc[df_vc_utmsource['source'], 'cat'][x]
    except:
        t = 0
    return t

In [4]:
%%time
adv_soc_net = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw','gVRrcxiDQubJiljoTbGm']
res['adv_soc_net'] = res['utm_source'].apply(lambda x: 'from_adv' if x in adv_soc_net else 'other_source')

range_source = [0, 500, 5000, 50000]
cat_source = [1, 2, 3]
vc_utmsource = res[res['target_event']==1]['utm_source'].value_counts()
df_vc_utmsource = pd.DataFrame(vc_utmsource)
df_vc_utmsource['source'] = df_vc_utmsource.index
df_vc_utmsource = df_vc_utmsource.rename(columns={'utm_source': 'count'})
df_vc_utmsource['cat'] = pd.cut(df_vc_utmsource['count'], range_source, labels=cat_source).astype(np.int8)

res['cat_source'] = res['utm_source'].apply(lambda x: foo_source(x))
# res = res.drop(['utm_source'], axis=1)

CPU times: total: 13min 39s
Wall time: 13min 41s


### utm_medium

In [5]:
def foo_medium(x):
    #print(x)
    try:
        t = df_vc_utmmedium.loc[df_vc_utmmedium['medium'], 'cat'][x]
    except:
        t = 0
    return t

In [6]:
%%time
utm_medium_type = ['organic', 'referral', '(none)']
res['utm_medium_type'] = res['utm_medium'].apply(lambda x: 'organic' if x in ['organic', 'referral', '(none)'] else 'paid_traffic')

range_medium = [0, 20, 200, 2000, 20000]
cat_medium = [0, 1, 2, 3]
vc_utmmedium = res[res['target_event']==1]['utm_medium'].value_counts()
df_vc_utmmedium = pd.DataFrame(vc_utmmedium)
df_vc_utmmedium['medium'] = df_vc_utmmedium.index
df_vc_utmmedium = df_vc_utmmedium.rename(columns={'utm_medium': 'count'})
df_vc_utmmedium['cat'] = pd.cut(df_vc_utmmedium['count'], range_medium, labels=cat_medium)

res['cat_medium'] = res['utm_medium'].apply(lambda x: foo_medium(x))
# res = res.drop(['utm_medium'], axis=1)

CPU times: total: 14min 6s
Wall time: 14min 9s


### utm_adcontent

In [7]:
def foo_adcontent(x):
    #print(x)
    try:
        t = df_vc_utmmedium.loc[df_vc_utmmedium['medium'], 'cat'][x]
    except:
        t = 0
    return t

In [8]:
%%time
range_adcontent = [0, 100, 1000, 10000, 100000]
cat_adcontent = [0, 1, 2, 3]
vc_adcontent = res[res['target_event']==1]['utm_adcontent'].value_counts()
df_vc_adcontent = pd.DataFrame(vc_adcontent)
df_vc_adcontent['adcontent'] = df_vc_adcontent.index
df_vc_adcontent = df_vc_adcontent.rename(columns={'utm_adcontent': 'count'})
df_vc_adcontent['cat'] = pd.cut(df_vc_adcontent['count'], range_adcontent, labels=cat_adcontent)
res['cat_adcontent'] = res['utm_adcontent'].apply(lambda x: foo_adcontent(x))
# res = res.drop(['utm_adcontent'], axis=1)

CPU times: total: 13min 6s
Wall time: 13min 8s


### utm_campaign

In [9]:
def foo_camp(x):
    #print(x)
    try:
        t = df_vc_utmmedium.loc[df_vc_utmmedium['medium'], 'cat'][x]
    except:
        t = 0
    return t

In [10]:
%%time
range_campaign = [0, 100, 1000, 10000, 100000]
cat_camp = [0, 1, 2, 3]
vc_camp = res[res['target_event']==1]['utm_campaign'].value_counts()
df_vc_camp = pd.DataFrame(vc_camp)
df_vc_camp['campaign'] = df_vc_camp.index
df_vc_camp = df_vc_camp.rename(columns={'utm_campaign': 'count'})
df_vc_camp['cat'] = pd.cut(df_vc_camp['count'], range_campaign, labels=cat_camp)
res['cat_camp'] = res['utm_campaign'].apply(lambda x: foo_camp(x))
# res = res.drop(['utm_campaign'], axis=1)

CPU times: total: 13min 10s
Wall time: 13min 12s


In [11]:
res = res.drop(['session_id', 'client_id', 'device_model', 'visit_number', 'utm_source', 'utm_medium', 'utm_adcontent', 'utm_campaign', 'utm_keyword'], axis=1)
res.head(10)

,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_event,cat_hour,cat_month,adv_soc_net,cat_source,utm_medium_type,cat_medium,cat_adcontent,cat_camp
0,mobile,Android,Huawei,360x720,Chrome,Russia,Zlatoust,0,4,3,other_source,3,paid_traffic,3,0,0
1,mobile,Android,Samsung,385x854,Samsung Internet,Russia,Moscow,0,2,3,from_adv,2,paid_traffic,3,0,0
2,mobile,Android,Huawei,360x720,Chrome,Russia,Krasnoyarsk,0,1,4,other_source,3,paid_traffic,3,0,0
3,mobile,unknown,Xiaomi,393x786,Chrome,Russia,Moscow,0,1,3,other_source,3,paid_traffic,3,0,0
4,mobile,unknown,Xiaomi,393x786,Chrome,Russia,Moscow,0,1,3,other_source,3,paid_traffic,3,0,0
5,mobile,unknown,Apple,375x812,Safari,Russia,Saint Petersburg,0,4,3,other_source,3,organic,3,0,0
6,tablet,Android,Lenovo,602x1029,YaBrowser,Russia,Saint Petersburg,0,2,4,other_source,0,paid_traffic,3,0,0
7,mobile,unknown,Samsung,360x640,Chrome,Russia,Saint Petersburg,0,3,1,other_source,3,paid_traffic,3,0,0
8,mobile,iOS,Apple,390x844,Safari,Russia,Moscow,0,4,4,other_source,3,paid_traffic,3,0,0
9,mobile,unknown,Samsung,412x915,Chrome,Russia,Moscow,0,4,3,other_source,3,paid_traffic,3,0,0


## DEVICE_*

### device_category

device_category - 3 категории, пропусков нет. Из преобразований - будет последующее onehot-кодирование

### device_os, device_brand, device_browser

восстановим ОС из других значений device_*

In [12]:
best_brands = ['Samsung', 'Xiaomi', 'Huawei', 'POCO']

dev_types = ['mobile', 'tablet']

android_brands = ['Xiaomi', 'Samsung', 'Huawei', 'Vivo', 'LG',
       'BQ', 'OnePlus', 'itel', 'Asus', 'Sony', 'Android', 'OPPO',
       'Nokia', 'Realme', 'Alcatel', 'Infinix', 'Micromax', 'Blackview',
       'Oukitel', 'ZTE', 'Motorola', 'HOMTOM', 'Google', 'Lenovo',
       'China Phone', 'Neffos', 'Tecno', 'DOOGEE', 'Vsmart', 'Meizu', 'Umidigi', 'Leagoo', 'Highscreen', 'Ulefone', 'HTC',
       'Cubot', 'Symphony', 'Vestel', 'Guophone', 'Wileyfox', 'Vertex',
       'POCO', 'TCL', 'SonyEricsson', 'Nomu', 'Prestigio',
       'Philips', 'LeEco', 'DEXP', 'Hisense', 'Vernee', 'Tele2', 'CAT',
       'H96', 'Kyocera', 'Inoi', 'Acer', 'Kingplay', 'Sharp',
       'MTC', 'iNew', 'Karbonn', 'Wiko', 'Gionee', 'Digma',
       'Yota Devices', 'Kata', 'Poptel', 'Flylion', 'AGM', 'Teclast',
       'Fly', 'MLS', 'LeTV', 'Nuu', 'Celkon', 'Chuwi', 'Walton',
       'Haier', 'Coolpad', 'Xiaolajiao', 'Lava', 'General Mobile',
       'Black Fox', 'Condor', 'Razer', 'Microsoft', 'Fujitsu',
       'Alldocube', 'ZOJI', 'Amazon', 'PPTV', 'Jiake', 'Mito', 'Oysters',
       'NOA', 'Caterpillar', 'SenseIT', 'Verizon', 'Komu', 'BLU', 'RCA',
       'Cube', 'AT&T', 'Tonbux', 'Dark', 'Elephone', 'T-Mobile',
       'Smartfren', 'Billion', 'Gome', 'A1', 'MTS', 'Geotel', 'BLUBOO',
       'Ananda', 'MediaTek', 'KingSing', 'Ark', 'Essential',
       'Dragon Touch', 'E&L', 'Fero',  'Wigor', 'Tanix',
       'Mlais', 'InFocus', 'Smartisan', 'Beelink', 'Unihertz', 'Orbic',
       'Honeywell', 'Archos', 'RED', 'Sonim', 'Jiayu', 'Evertek',
       'Leegoog', 'QMobile', 'INUI', 'Artel', 'Irbis', 'China Mobile',
       'Flexymove', 'Olla', 'Krip', 'Star', 'UMI', 'Iris', 'Ellipsis',
       'LTC', 'Motive', 'Advan', 'M-HORSE', 'Winnovo', 'ThL', 'Nomi',
       'Wings Mobile', 'MXQ', 'How', 'Centric', 'Maze', 'Corn',
       'China TVBox', 'Gretel', 'Vertu', 'Evercoss', 'teXet', 'Qbex',
       'Panasonic', 'Tagital', 'Dell', 'Maxvi']

In [13]:
%%time
def foo(x):
    if x['device_os']=='unknown':
        if x['device_brand'] in android_brands:
            return 'Android'
        elif (x['device_brand'] == 'Apple') and (x.device_category in dev_types):
            return 'iOS'
    elif x['device_os'] != x['device_os']:
        if (x['device_category'] in dev_types) and (x['device_brand']=='Apple'):
            return 'iOS'
        elif x['device_brand'] in android_brands:
            return 'Android'
    else:
        return x['device_os']
    return x['device_os']
    
res['device_os'] = res.apply(lambda x: foo(x), axis=1)

CPU times: total: 22.8 s
Wall time: 22.9 s


device_brand преобразуем в три категории: Apple - 3, Samsung, Xiaomi, Huawei и Poco - 2, остальные бренды - 1
признак device_browser отбросим - 

In [14]:
def brand2cat(x):
    if x == 'Apple':
        return int(3)
    elif x in best_brands:
        return int(2)
    else:
        return int(1)
    return int(0)

res['cat_device_brand'] = res['device_brand'].apply(lambda x: brand2cat(x))
res = res.drop(['device_brand'], axis=1)

In [15]:
def foo_browser(x):
    #print(x)
    t = 0
    try:
        t = df_vc_browser.loc[df_vc_browser['browser'], 'cat'][x]
    except:
        t = 0
    return t

In [16]:
%%time
range_browser = [0, 60, 600, 6000, 60000]
cat_browser = [1, 2, 3, 4]
vc_browser = res[res['target_event']==1]['device_browser'].value_counts()
df_vc_browser = pd.DataFrame(vc_browser)
df_vc_browser['browser'] = df_vc_browser.index
df_vc_browser = df_vc_browser.rename(columns={'device_browser': 'count'})
df_vc_browser['cat'] = pd.cut(df_vc_browser['count'], range_browser, labels=cat_browser)
res['cat_browser'] = res['device_browser'].apply(lambda x: foo_browser(x)) #.astype(np.int8)
# res = res.drop(['utm_campaign'], axis=1)

CPU times: total: 12min 43s
Wall time: 12min 44s


In [17]:
def foo_resolution(x):
    #print(x)
    t = 0
    try:
        t = df_vc_resolution.loc[df_vc_browser['resolution'], 'cat'][x]
    except:
        t = 0
    return t

In [18]:
%%time
range_resolution = [0, 50, 500, 5000]
cat_resolution = [0, 1, 2]
vc_resolution = res[res['target_event']==1]['device_screen_resolution'].value_counts()
df_vc_resolution = pd.DataFrame(vc_resolution)
df_vc_resolution['resolution'] = df_vc_resolution.index
df_vc_resolution = df_vc_resolution.rename(columns={'device_screen_resolution': 'count'})
df_vc_resolution['cat'] = pd.cut(df_vc_resolution['count'], range_resolution, labels=cat_resolution)
res['cat_resolution'] = res['device_screen_resolution'].apply(lambda x: foo_resolution(x)) #.astype(np.int8)
# res = res.drop(['utm_campaign'], axis=1)

CPU times: total: 10.3 s
Wall time: 10.4 s


In [19]:
res = res.drop(['device_browser', 'device_screen_resolution'], axis=1)

## GEO_*
### geo_country

### geo_city

In [20]:
def foo_city(x):
    #print(x)
    t = 0
    try:
        t = df_vc_city.loc[df_vc_browser['city'], 'cat'][x]
    except:
        t = 0
    return t

In [21]:
%%time
range_city = [0, 100, 1000, 10000, 100000]
cat_city = [0, 1, 2, 3]
vc_city = res[res['target_event']==1]['geo_city'].value_counts()
df_vc_city = pd.DataFrame(vc_city)
df_vc_city['city'] = df_vc_city.index
df_vc_city = df_vc_city.rename(columns={'geo_city': 'count'})
df_vc_city['cat'] = pd.cut(df_vc_city['count'], range_city, labels=cat_city)
res['cat_city'] = res['geo_city'].apply(lambda x: foo_city(x)) #.astype(np.int8)
# res = res.drop(['utm_campaign'], axis=1)

CPU times: total: 9.25 s
Wall time: 9.25 s


In [22]:
res = res.drop(['geo_country', 'geo_city'], axis=1)

In [23]:
def view_occupancy(df):
    length = len(df)
    dic = dict()
    for col in df.columns:
        percent_filled = (length-df[col].isna().sum())/length
        dic[col] = percent_filled

    dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    for k, v in dic:
        print('{}: {:.3f}%'.format(k, v*100))

In [24]:
view_occupancy(res)

device_category: 100.000%
device_os: 100.000%
target_event: 100.000%
cat_hour: 100.000%
cat_month: 100.000%
adv_soc_net: 100.000%
cat_source: 100.000%
utm_medium_type: 100.000%
cat_medium: 100.000%
cat_adcontent: 100.000%
cat_camp: 100.000%
cat_device_brand: 100.000%
cat_browser: 100.000%
cat_resolution: 100.000%
cat_city: 100.000%


In [25]:
res.head()

,device_category,device_os,target_event,cat_hour,cat_month,adv_soc_net,cat_source,utm_medium_type,cat_medium,cat_adcontent,cat_camp,cat_device_brand,cat_browser,cat_resolution,cat_city
0,mobile,Android,0,4,3,other_source,3,paid_traffic,3,0,0,2,4,0,0
1,mobile,Android,0,2,3,from_adv,2,paid_traffic,3,0,0,2,3,0,0
2,mobile,Android,0,1,4,other_source,3,paid_traffic,3,0,0,2,4,0,0
3,mobile,Android,0,1,3,other_source,3,paid_traffic,3,0,0,2,4,0,0
4,mobile,Android,0,1,3,other_source,3,paid_traffic,3,0,0,2,4,0,0


In [28]:
# res = res.sample(frac=1).reset_index(drop=True)
res = res.sample(n=200000).reset_index(drop=True)

cols_for_ohe = ['device_category', 'device_os', 'adv_soc_net', 'utm_medium_type']

ohe = OneHotEncoder(sparse=False)

ohe.fit(res[cols_for_ohe])
transformed = ohe.transform(res[cols_for_ohe])
res[ohe.get_feature_names_out()] = transformed
df_ohed = res.drop(cols_for_ohe, axis=1)

%store df_ohed

Stored 'df_ohed' (DataFrame)
